In [1]:
import plotly.express as px
import os
import sys
import numpy as np
import scipy.stats as stats

%load_ext autoreload
%autoreload 2

In [2]:
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from experiments.adversarial_filtering.partial_input_baselines.partial_input_result_buckets import partial_input_human
from experiments.result_buckets import roberta_specialized
from experiments.bucket_analysis import BucketDatasetResult

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:00<00:00, 36719.99it/s]


In [8]:
from simple_colors import * 
for dataset in ['snli', 'atomic', 'social']:
    print(green(dataset))
    artifacts_ids = []
    non_artifacts_ids  = []
    
    for b in partial_input_human[f'{dataset}-human'].buckets:
        if b.original_example_prediction.correct:
            artifacts_ids.append(b.original_example_id)
        else:
            non_artifacts_ids.append(b.original_example_id)

    artifacts = BucketDatasetResult(
        [b for b in roberta_specialized[f'{dataset}-human'].buckets if b.original_example_id in artifacts_ids]
    )
    no_artifacts = BucketDatasetResult(
        [b for b in roberta_specialized[f'{dataset}-human'].buckets if b.original_example_id in non_artifacts_ids]
    )
    print('Num examples with artifacts:', len(artifacts_ids), '\nNum examples without artifacts:', len(non_artifacts_ids))

    print('Artifact Examples:\n', artifacts.linguistic_robustness_summary(roberta_specialized[f'{dataset}-test'])['stay_prob_corrected'])
    print()
    print('Non-artifact Examples:\n', no_artifacts.linguistic_robustness_summary(roberta_specialized[f'{dataset}-test'])['stay_prob_corrected'])
    print()
    print('Whole dataset:\n', roberta_specialized[f'{dataset}-human'].linguistic_robustness_summary(roberta_specialized[f'{dataset}-test'])['stay_prob_corrected'])
    print()

snli
Num examples with artifacts: 162 
Num examples without artifacts: 88
Artifact Examples:
 0.9136816394956326

Non-artifact Examples:
 0.840066176960795

Whole dataset:
 0.9009562634337458

atomic
Num examples with artifacts: 144 
Num examples without artifacts: 106
Artifact Examples:
 0.8738089920171768

Non-artifact Examples:
 0.8647201672184828

Whole dataset:
 0.8709695949924339

social
Num examples with artifacts: 136 
Num examples without artifacts: 114
Artifact Examples:
 0.9087786432543185

Non-artifact Examples:
 0.9348597589038101

Whole dataset:
 0.9193559020581005

